<a href="https://colab.research.google.com/github/omnidox/cognizant-genspark-genai-training/blob/main/prompt-engineering-llms/deliverables/mod_4_Fine-Tuning_LLMs/assignments/fine_tuning_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install torch tensorflow transformers datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Check GPU availability
import torch
print("GPU Available:", torch.cuda.is_available())

GPU Available: True


In [4]:
# Load and preprocess the IMDB dataset
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
# Load Pretrained DistilBERT Model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Check GPU availability
import torch
print("GPU Available:", torch.cuda.is_available())

GPU Available: True


In [18]:
# Define Training Arguments and Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Use `eval_strategy` instead of deprecated `evaluation_strategy`
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=200,
    report_to="none",  # 👈 Disables W&B and other logging integrations
    fp16=True,  # Enables mixed-precision training
)

# Limit dataset size
train_subset = tokenized_dataset["train"].shuffle(seed=42).select(range(5_000))  # Use only 5,000 training samples
eval_subset = tokenized_dataset["test"].shuffle(seed=42).select(range(1_000))  # Use only 1,000 test samples


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,  # Use the smaller dataset
    eval_dataset=eval_subset,
)


In [19]:
# Fine-Tune the Model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.248061
2,No log,0.272539
3,0.180300,0.293641


TrainOutput(global_step=237, training_loss=0.16818630443846627, metrics={'train_runtime': 194.7386, 'train_samples_per_second': 77.026, 'train_steps_per_second': 1.217, 'total_flos': 1987010979840000.0, 'train_loss': 0.16818630443846627, 'epoch': 3.0})

In [20]:
# Save the Fine-Tuned Model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [22]:
# Detailed Metrics Evaluation
from sklearn.metrics import classification_report
import numpy as np

predictions = trainer.predict(tokenized_dataset["test"])
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = tokenized_dataset["test"]["label"]

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91     12500
           1       0.91      0.91      0.91     12500

    accuracy                           0.91     25000
   macro avg       0.91      0.91      0.91     25000
weighted avg       0.91      0.91      0.91     25000



In [31]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.3 MB/s eta 0:00:00


In [34]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")  # ✅ Corrected version


In [35]:
import evaluate
import numpy as np

# Load accuracy metric
metric = evaluate.load("accuracy")

# Define function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Pass `compute_metrics` to Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics  # ✅ Now accuracy will be included in results
)


In [36]:
results = trainer.evaluate()
print(results)


{'eval_loss': 0.2936406433582306, 'eval_model_preparation_time': 0.0043, 'eval_accuracy': 0.9, 'eval_runtime': 4.5265, 'eval_samples_per_second': 220.922, 'eval_steps_per_second': 3.535}


In [37]:
# Reflection & Suggestions for Improvement
if results['eval_accuracy'] < 0.90:
    print("\nSuggested Improvements:")
    print("- Increase the number of training epochs.")
    print("- Experiment with different learning rates.")
    print("- Use data augmentation techniques to improve generalization.")
    print("- Fine-tune on a larger dataset for better accuracy.")